In [1]:
import pandas as pd 
import numpy as np
import datetime
import pytz

In [2]:
df = pd.read_csv('Tesla_Comments_filtered.csv', delimiter=";")


C:\Users\maart\AppData\Local\Temp\ipykernel_31384\369132780.py:1: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Tesla_Comments_filtered.csv', delimiter=";")


In [3]:
df = df[['subreddit','id','body','created_utc','parent_id']]


In [4]:
# Convert "body" column to integers
df['created_utc'] = pd.to_numeric(df['created_utc'], errors='coerce')
df = df.dropna(subset=['created_utc'])

# Define function to convert Unix timestamp to datetime object
def unix_to_datetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp)

# Apply function to Unix timestamp column
df['date'] = df['created_utc'].apply(unix_to_datetime)

C:\Users\maart\AppData\Local\Temp\ipykernel_31384\2494881415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_utc'] = pd.to_numeric(df['created_utc'], errors='coerce')
C:\Users\maart\AppData\Local\Temp\ipykernel_31384\2494881415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['created_utc'].apply(unix_to_datetime)


In [5]:
print(df['body'].isnull().sum())

20149


In [6]:
df.dropna(inplace=True)

C:\Users\maart\AppData\Local\Temp\ipykernel_31384\1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [7]:
# #SOLUTION 1
# from pysentimiento import create_analyzer
# emotion_analyzer = create_analyzer(task="sentiment", lang="en")

c:\Users\maart\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Using the emotion analyer on all reddit comments and put it under a new column 
# df['emotion'] = df['body'].apply(emotion_analyzer.predict)


C:\Users\maart\AppData\Local\Temp\ipykernel_31384\1505528588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emotion'] = df['body'].apply(emotion_analyzer.predict)


In [17]:
#This was done to list the probabilities of the analyzeroutput
# df['probas'] = df['emotion'].apply(lambda x: list(x.probas.items()))


C:\Users\maart\AppData\Local\Temp\ipykernel_31384\2427439957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['probas'] = df['emotion'].apply(lambda x: list(x.probas.items()))


In [23]:
#This was done to get a score that can be used in the next step to match frequencies with stock prices

calc_reg_output = lambda row: -1 * row['probas'][0][1] + 0 * row['probas'][1][1] + 1 * row['probas'][2][1]


In [25]:
#Now we apply the above function to every single row so that we have one score for every comment
df['reg_output'] = df.apply(calc_reg_output, axis=1)


C:\Users\maart\AppData\Local\Temp\ipykernel_31384\624689051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reg_output'] = df.apply(calc_reg_output, axis=1)


In [26]:
df

,subreddit,id,body,created_utc,parent_id,date,emotion,probas,reg_output
0,TeslaMotors,hqqtbh2,"If help is needed, use our stickied support th...",1.640994e+09,t3_rt6fwg,2022-01-01 00:46:50,"AnalyzerOutput(output=NEU, probas={NEU: 0.832,...","[(NEG, 0.0038293171674013138), (NEU, 0.8324310...",0.159910
1,TeslaMotors,hqqtz7i,Requirements: \n\n• Fits under the aero cover ...,1.640995e+09,t3_rt6j33,2022-01-01 00:51:46,"AnalyzerOutput(output=NEU, probas={NEU: 0.872,...","[(NEG, 0.11134149879217148), (NEU, 0.871991336...",-0.094674
2,TeslaMotors,hqqu7z3,Photo credit @rkaplan1 on Twitter,1.640995e+09,t3_rt6kex,2022-01-01 00:53:37,"AnalyzerOutput(output=NEU, probas={NEU: 0.922,...","[(NEG, 0.003630931256338954), (NEU, 0.92183905...",0.070899
3,TeslaMotors,hqqxko8,I've wanted an Apple MagSafe charger in my cen...,1.640996e+09,t3_rt72vs,2022-01-01 01:20:13,"AnalyzerOutput(output=NEG, probas={NEG: 0.785,...","[(NEG, 0.784606397151947), (NEU, 0.20615413784...",-0.775367
4,TeslaMotors,hqr8kka,I see you set your temperature to: NICE.,1.641002e+09,t3_rt8o5j,2022-01-01 02:47:11,"AnalyzerOutput(output=POS, probas={POS: 0.675,...","[(NEG, 0.00296275457367301), (NEU, 0.322249382...",0.671825
...,...,...,...,...,...,...,...,...,...
168121,TeslaMotors,ie4b81w,Is this with the unreleased beta software? I’m...,1.656459e+09,t3_vn0xq2,2022-06-29 01:24:55,"AnalyzerOutput(output=NEG, probas={NEG: 0.778,...","[(NEG, 0.7782235741615295), (NEU, 0.2154842764...",-0.771931
168122,TeslaMotors,ie4bcyt,No still 10.12.2. Just in another vehicle.,1.656459e+09,t3_vn0yi6,2022-06-29 01:25:58,"AnalyzerOutput(output=NEU, probas={NEU: 0.973,...","[(NEG, 0.015009327791631222), (NEU, 0.97261792...",-0.002637
168123,TeslaMotors,ie4betk,"Also wondering this question, would love to re...",1.656459e+09,t3_vn0ysw,2022-06-29 01:26:22,"AnalyzerOutput(output=NEU, probas={NEU: 0.719,...","[(NEG, 0.0524548701941967), (NEU, 0.7192255258...",0.175865
168124,TeslaMotors,ie4nqju,Looks like a bunch of companies are all releas...,1.656465e+09,t3_vn0ysw,2022-06-29 03:03:36,"AnalyzerOutput(output=NEU, probas={NEU: 0.903,...","[(NEG, 0.005526799242943525), (NEU, 0.90288501...",0.086061


In [28]:
#This is done to save the output of the analyzeroutput, so that above code does not have to be run again
df.to_csv('Tesla_Comments_Filtered_Analyzed.csv', index=False)